# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 5 2022 10:49AM,241062,20,ALU0007193,Alumier Labs Inc.,Released
1,Jul 5 2022 10:49AM,241062,20,ALU0007194,Alumier Labs Inc.,Released
2,Jul 5 2022 10:49AM,241062,20,ALUR625199632,Alumier Labs Inc.,Released
3,Jul 5 2022 10:49AM,241062,20,ALU0007195,Alumier Labs Inc.,Released
4,Jul 5 2022 10:49AM,241062,20,ALU0007196,Alumier Labs Inc.,Released
5,Jul 5 2022 10:49AM,241062,20,ALUR668841103,Alumier Labs Inc.,Released
6,Jul 5 2022 10:49AM,241062,20,ALU0007197,Alumier Labs Inc.,Released
7,Jul 5 2022 10:49AM,241062,20,ALU0007198,Alumier Labs Inc.,Released
8,Jul 5 2022 10:49AM,241062,20,ALUR004150114,Alumier Labs Inc.,Released
9,Jul 5 2022 10:49AM,241062,20,ALUR089276022,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
56,241075,Released,3
57,241076,Released,9
58,241077,Released,1
59,241078,Released,2
60,241079,Released,9


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241075,NaN,3.0
241076,NaN,9.0
241077,NaN,1.0
241078,NaN,2.0
241079,NaN,9.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241004,5.0,54.0
241005,0.0,54.0
241006,0.0,55.0
241009,0.0,60.0
241010,0.0,60.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241004,5,54
241005,0,54
241006,0,55
241009,0,60
241010,0,60


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241004,5,54
1,241005,0,54
2,241006,0,55
3,241009,0,60
4,241010,0,60


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241004,5,54
1,241005,,54
2,241006,,55
3,241009,,60
4,241010,,60


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 5 2022 10:49AM,241062,20,Alumier Labs Inc.
29,Jul 5 2022 10:46AM,241079,10,ISDIN Corporation
38,Jul 5 2022 10:43AM,241078,10,ISDIN Corporation
40,Jul 5 2022 10:32AM,241076,10,MedStone Pharma LLC
49,Jul 5 2022 10:32AM,241077,10,"Nextsource Biotechnology, LLC"
50,Jul 5 2022 10:30AM,241075,10,Emerginnova
53,Jul 5 2022 10:30AM,241073,15,"Virtus Pharmaceuticals, LLC"
72,Jul 5 2022 10:28AM,241066,15,"Person & Covey, Inc."
79,Jul 5 2022 10:26AM,241074,10,ISDIN Corporation
120,Jul 5 2022 10:26AM,241070,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 5 2022 10:49AM,241062,20,Alumier Labs Inc.,,29
1,Jul 5 2022 10:46AM,241079,10,ISDIN Corporation,,9
2,Jul 5 2022 10:43AM,241078,10,ISDIN Corporation,,2
3,Jul 5 2022 10:32AM,241076,10,MedStone Pharma LLC,,9
4,Jul 5 2022 10:32AM,241077,10,"Nextsource Biotechnology, LLC",,1
5,Jul 5 2022 10:30AM,241075,10,Emerginnova,,3
6,Jul 5 2022 10:30AM,241073,15,"Virtus Pharmaceuticals, LLC",,19
7,Jul 5 2022 10:28AM,241066,15,"Person & Covey, Inc.",,7
8,Jul 5 2022 10:26AM,241074,10,ISDIN Corporation,,41
9,Jul 5 2022 10:26AM,241070,10,ISDIN Corporation,,49


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 10:49AM,241062,20,Alumier Labs Inc.,29,
1,Jul 5 2022 10:46AM,241079,10,ISDIN Corporation,9,
2,Jul 5 2022 10:43AM,241078,10,ISDIN Corporation,2,
3,Jul 5 2022 10:32AM,241076,10,MedStone Pharma LLC,9,
4,Jul 5 2022 10:32AM,241077,10,"Nextsource Biotechnology, LLC",1,
5,Jul 5 2022 10:30AM,241075,10,Emerginnova,3,
6,Jul 5 2022 10:30AM,241073,15,"Virtus Pharmaceuticals, LLC",19,
7,Jul 5 2022 10:28AM,241066,15,"Person & Covey, Inc.",7,
8,Jul 5 2022 10:26AM,241074,10,ISDIN Corporation,41,
9,Jul 5 2022 10:26AM,241070,10,ISDIN Corporation,49,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 10:49AM,241062,20,Alumier Labs Inc.,29,
1,Jul 5 2022 10:46AM,241079,10,ISDIN Corporation,9,
2,Jul 5 2022 10:43AM,241078,10,ISDIN Corporation,2,
3,Jul 5 2022 10:32AM,241076,10,MedStone Pharma LLC,9,
4,Jul 5 2022 10:32AM,241077,10,"Nextsource Biotechnology, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 10:49AM,241062,20,Alumier Labs Inc.,29.0,NaN
1,Jul 5 2022 10:46AM,241079,10,ISDIN Corporation,9.0,NaN
2,Jul 5 2022 10:43AM,241078,10,ISDIN Corporation,2.0,NaN
3,Jul 5 2022 10:32AM,241076,10,MedStone Pharma LLC,9.0,NaN
4,Jul 5 2022 10:32AM,241077,10,"Nextsource Biotechnology, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 10:49AM,241062,20,Alumier Labs Inc.,29.0,0.0
1,Jul 5 2022 10:46AM,241079,10,ISDIN Corporation,9.0,0.0
2,Jul 5 2022 10:43AM,241078,10,ISDIN Corporation,2.0,0.0
3,Jul 5 2022 10:32AM,241076,10,MedStone Pharma LLC,9.0,0.0
4,Jul 5 2022 10:32AM,241077,10,"Nextsource Biotechnology, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,11087876,1658.0,82.0
12,723140,76.0,6.0
15,1446390,103.0,0.0
19,241057,1.0,0.0
20,241062,29.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,11087876,1658.0,82.0
1,12,723140,76.0,6.0
2,15,1446390,103.0,0.0
3,19,241057,1.0,0.0
4,20,241062,29.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,1658.0,82.0
1,12,76.0,6.0
2,15,103.0,0.0
3,19,1.0,0.0
4,20,29.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1658.0
1,12,Released,76.0
2,15,Released,103.0
3,19,Released,1.0
4,20,Released,29.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20
Status,,,,,
Executing,82.0,6.0,0.0,0.0,0.0
Released,1658.0,76.0,103.0,1.0,29.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20
0,Executing,82.0,6.0,0.0,0.0,0.0
1,Released,1658.0,76.0,103.0,1.0,29.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20
0,Executing,82.0,6.0,0.0,0.0,0.0
1,Released,1658.0,76.0,103.0,1.0,29.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()